In [ ]:
#this query brings togather all mace queries into one to union them into a table where analysis can be performed 
create or replace table `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_mace` as (

#first cardiovascular deaths 
with cv_death as (
  with deaths_carehome as(
select 
*
from yhcr-prd-bradfor-bia-core.CB_FDM_DeathCertificates.tbl_NEC_Deaths
where person_id in (select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` )),

#left join ch_admission date 
death_carehome_merge as (
  select
dch.person_id, tbl_NEC_Deaths_end_date,dod,reg_date,s_cod_code_1,S_COD_LINE_1,s_cod_code_4,
  ch.first_episodestartdate
  from deaths_carehome dch
  left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` ch
  on dch.person_id = ch.person_id
),

#got 3 duplicate ntries for a lot of people, there doesn't seem to be any difference between the duplicate excpet that some are missing the dod and reg date, therefore i have used a row_number function to select the duplicate htat contains a dod and reg_date 
dch_rank as (
select
*,
row_number() over (partition by person_id order by dod, reg_date desc) as duplicate_rank
 from 
death_carehome_merge),

duplicate_remove  as(
select
person_id, tbl_NEC_Deaths_end_date as dod,s_cod_code_1,first_episodestartdate
from dch_rank
where (duplicate_rank = 1) and ((s_cod_code_1 like 'I%') OR (s_cod_code_1 like '%irculat%') or (s_cod_code_4 like '%irculatory%')) and (tbl_NEC_Deaths_end_date > (EXTRACT(date from first_episodestartdate))))

select
person_id,duplicate_remove.dod, case when person_id is not null then "cv death" end as event_type
from duplicate_remove 
where date_diff(duplicate_remove.dod, (EXTRACT(date from first_episodestartdate)),day) <366
),

#then myocardial infarction as 

mi as (
  #airedale
  with mi_aire as  
(
with
episode_ch_cohort_2019 as (
  select 
person_id, 
tbl_SUS_Airedale_APC_20190201_to_20200630_start_date as episode_start, 
tbl_SUS_Airedale_APC_20190201_to_20200630_end_date as episode_end, 
cast(ep_no as INT64) as episode_sequence, 
rtrim(diag1,'X') as diagnosis_1, 
rtrim(diag2,'X') as diagnosis_2,
rtrim(diag3,'X') as diagnosis_3, 
from `yhcr-prd-bradfor-bia-core.CB_FDM_Airedale.tbl_SUS_Airedale_APC_20190201_to_20200630`
where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)
),

episode_ch_cohort_2020 as (
  select 
person_id, 
tbl_SUS_Airedale_APC_20200701_to_20220710_mrg_start_date as episode_start, 
tbl_SUS_Airedale_APC_20200701_to_20220710_mrg_end_date as episode_end, 
cast(ep_no as INT64) as episode_sequence, 
rtrim(diag1,'X') as diagnosis_1, 
rtrim(diag2,'X') as diagnosis_2,
rtrim(diag3,'X') as diagnosis_3, 
from `yhcr-prd-bradfor-bia-core.CB_FDM_Airedale.tbl_SUS_Airedale_APC_20200701_to_20220710_mrg`
where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)
),
#for some reason the airedale hospital records are split into prea nd post 2019 so i had to combine the two tables toether to ensure no episodes were missed 

episode_ch_cohort_combined as (


  select
  * 
  from episode_ch_cohort_2019

  union all

  select
  *
  from episode_ch_cohort_2020
),


#joins care home entry date from the master cohort table 
episode_entrydate_join as (
  select 
  e_c.*,
  a.first_episodestartdate
  from episode_ch_cohort_combined as e_c
  left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` a
  on e_c.person_id = a.person_id),
#filters out episodes which began after the admission date to a care home
episode_ch_filtered as (
  select
e_e_j.*
  from episode_entrydate_join e_e_j
  where (episode_start > first_episodestartdate) AND (datetime_diff(episode_start,first_episodestartdate, day) < 366)
),
#up to this point reprents the code to identify and clean alll seocndary inpatient episodes for indiivudals in the connected bradford cohort in the 12 months after admission,the next line of code will identify admission related to falls, will then union this with a table for falls in AE

#bradford 

mi_ch as (
select
*,
#this line ranks episodes according to the date they occurred for each person, this way we can identify the most recent hospital attendance following admission 
row_number() over (partition by person_id order by episode_start) as admission_seq
from episode_ch_filtered
where diagnosis_1 like "I21%" or diagnosis_1 like "I22%" or diagnosis_2 like "I21%" or diagnosis_2 like "I22%" or diagnosis_3 like "I21%" or diagnosis_3 like "I22%"
order by person_id)
#this then filters for the first hopsital spell with a fall coded in any of the diagnosis codes. it is theoretically possible that a patient had an inpatient fall which was coded in the co-moribidities rather than the primary reason for presntation, but it is assumed the margin of error is relatively small.

select
*
from mi_ch
where admission_seq = 1),

mi_bradford as (
  with episode_ch_cohort as (
select 
person_id, 
tbl_episode_start_date, 
tbl_episode_end_date, 
rtrim(diagnosis_1,'X') as diagnosis_1, 
rtrim(diagnosis_2,'X') as diagnosis_2,
rtrim(diagnosis_3,'X') as diagnosis_3, 
from `yhcr-prd-bradfor-bia-core.CB_FDM_BradfordRoyalInfirmary.tbl_episode`
where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` )),

#joins care home entry date from the master cohort table 
episode_entrydate_join as (
  select 
  e_c.*,
  a.first_episodestartdate
  from episode_ch_cohort as e_c
  left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` a
  on e_c.person_id = a.person_id),
#filters out episodes which began after the admission date to a care home
episode_ch_filtered as (
  select
e_e_j.*
  from episode_entrydate_join e_e_j
  where (tbl_episode_start_date > first_episodestartdate) AND (datetime_diff(tbl_episode_start_date, first_episodestartdate, day) < 366 )
),
#filter for the first episode in a spell and extract the first three characters from each icd10 diagnosis code (not interested in the fourth character code for falls code)
episode_ch_final as (
select 
person_id, 
diagnosis_1,
diagnosis_2,
diagnosis_3,
tbl_episode_start_date,
tbl_episode_end_date,   
#substr(diagnosis_1,1,3) as diagnosis_1_substr,
#substr(diagnosis_2,1,3) as diagnosis_2_substr,
#substr(diagnosis_3,1,3) as diagnosis_3_substr,
#substr(diagnosis_4,1,3) as diagnosis_4_substr,
#substr(diagnosis_5,1,3) as diagnosis_5_substr,
#substr(diagnosis_6,1,3) as diagnosis_6_substr,
#substr(diagnosis_7,1,3) as diagnosis_7_substr,
#substr(diagnosis_8,1,3) as diagnosis_8_substr,
#substr(diagnosis_9,1,3) as diagnosis_9_substr,
#substr(diagnosis_10,1,3) as diagnosis_10_substr,
#substr(diagnosis_11,1,3) as diagnosis_11_substr,
from episode_ch_filtered
order by person_id),
#up to this point reprents the code to identify and clean all seocndary inpatient episodes for indiivudals in the connected bradford cohort in the 12 months after admission,the next line of code will identify admission related to falls, will then union this with a table for falls in AE. admissions related to falls was defined as any admission with a SNOMED OR ICD10 code available. 

mi_ch as (
select
*,
row_number() over (partition by person_id order by tbl_episode_start_date) as admission_seq
from episode_ch_final 
where diagnosis_1 like "I21%" or diagnosis_1 like "I22%" or diagnosis_2 like "I21%" or diagnosis_2 like "I22%" or diagnosis_3 like "I21%" or diagnosis_3 like "I22%"
order by person_id)
#this then filters for the first hopsital spell with a fall coded in any of the diagnosis codes. it is theoretically possible that a patient had an inpatient fall which was coded in the co-moribdities rather than the primary reason for presntation, but it is assumed the margin of error is relatively small.

select
*,

from mi_ch
where admission_seq = 1
order by person_id),

#primary care 

mi_primary_care as 
(select
person_id,dateevent, ctv3code
from `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode` where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` ) and ctv3code in(select CTV3ID from `yhcr-prd-bradfor-bia-core.CB_MYSPACE_MK.myocardial_infarction_ctv3code` )),

mi_primary_care_join as (
select
mip.*,care_home.first_episodestartdate
from mi_primary_care mip
left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` care_home on mip.person_id = care_home.person_id ),

mi_primary_care_post_admission as (
  select
*
  from mi_primary_care_join
  where (dateevent > first_episodestartdate) and (date_diff(dateevent, first_episodestartdate,day) < 366)),

mi_union as (
select 
person_id,
tbl_episode_start_date
from mi_bradford

union all 

select
person_id, 
dateevent
from mi_primary_care_post_admission

union all 

select
person_id,
mi_aire.episode_start
from
mi_aire)

select
*, case when person_id is not null then "myocardial infarction" end as cv_event 
from mi_union
order by person_id),

#stroke

cva as (
with stroke as (
SELECT person_id, diagnosis_1, diagnosis_2,diagnosis_3, tbl_episode_end_date, episode_number FROM `yhcr-prd-bradfor-bia-core.CB_FDM_BradfordRoyalInfirmary.tbl_episode` where diagnosis_1 in(select icd from  `yhcr-prd-bradfor-bia-core.CB_MYSPACE_MK.icd10_stroke_code`) OR diagnosis_2 in(select icd from  `yhcr-prd-bradfor-bia-core.CB_MYSPACE_MK.icd10_stroke_code`) OR diagnosis_3 in(select icd from  `yhcr-prd-bradfor-bia-core.CB_MYSPACE_MK.icd10_stroke_code`)),
# first off we have bradford storkes cases (makes upp the majority) 

stroke_master_merged as (
select
stroke.*,
a.first_episodestartdate  as ch_entrydate
FROM stroke
LEFT JOIN `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` a 
on stroke.person_id = a.person_id),

stroke_master as(
SELECT 
*,
row_number() over (partition by person_id order by tbl_episode_end_date) diagnosis_seq
from stroke_master_merged
where datetime_diff(tbl_episode_end_date,ch_entrydate,day) < 366 AND stroke_master_merged.tbl_episode_end_date > stroke_master_merged.ch_entrydate
order by person_id),

stroke_bthft as (
SELECT 
*
from stroke_master
where diagnosis_seq = 1),

#next we have calderdale cases 

stroke_cld as (
SELECT 
person_id,
tbl_SUS_Calderdale_APC_CM_20170618_To_20220731_end_date as admission_end_date,
tbl_SUS_Calderdale_APC_CM_20170618_To_20220731_start_date as admission_start_date,
diagnosis_primary_icd, 
diagnosis_1st_secondary_icd, 
diagnosis_2nd_secondary_icd,
diagnosis_3rd_secondary_icd,
diagnosis_4th_secondary_icd,
diagnosis_5th_secondary_icd,
diagnosis_6th_secondary_icd,
diagnosis_7th_secondary_icd
FROM `yhcr-prd-bradfor-bia-core.CB_FDM_Calderdale.tbl_SUS_Calderdale_APC_CM_20170618_To_20220731` 
where episode_number = '1' and (
diagnosis_primary_icd in(select icd from  `yhcr-prd-bradfor-bia-core.CB_MYSPACE_MK.icd10_stroke_code`)
OR diagnosis_1st_secondary_icd in(select icd from  `yhcr-prd-bradfor-bia-core.CB_MYSPACE_MK.icd10_stroke_code`) 
OR diagnosis_2nd_secondary_icd in(select icd from  `yhcr-prd-bradfor-bia-core.CB_MYSPACE_MK.icd10_stroke_code`) 
OR diagnosis_3rd_secondary_icd in(select icd from  `yhcr-prd-bradfor-bia-core.CB_MYSPACE_MK.icd10_stroke_code`) 
OR diagnosis_4th_secondary_icd in(select icd from  `yhcr-prd-bradfor-bia-core.CB_MYSPACE_MK.icd10_stroke_code`) 
OR diagnosis_5th_secondary_icd in(select icd from  `yhcr-prd-bradfor-bia-core.CB_MYSPACE_MK.icd10_stroke_code`))),

stroke_master_merged_cld as (
select
stroke_cld.*,
a.first_episodestartdate  as ch_entrydate
FROM stroke_cld
LEFT JOIN `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` a 
on stroke_cld.person_id = a.person_id),

stroke_final_cld as(
SELECT 
*,
row_number() over (partition by person_id order by admission_end_date) diagnosis_seq
from stroke_master_merged_cld
where datetime_diff(admission_end_date,ch_entrydate,day) < 366 AND (stroke_master_merged_cld.admission_end_date > stroke_master_merged_cld.ch_entrydate)

order by person_id),

stroke_calderdale as (
SELECT 
*
from stroke_final_cld
where diagnosis_seq = 1),

# now we union both together
stroke_bthft_chft as (
select person_id, tbl_episode_end_date as stroke_diagnosis_date,case when person_id is not  null then "stroke" end as cv_event
from stroke_bthft

UNION DISTINCT

select person_id, admission_start_date as stroke_diagnosis_date,case when person_id is not  null then "stroke" end as cv_event
from stroke_calderdale)

select
* from stroke_bthft_chft
),

union_mace as (
select
* from cv_death

union  all

select
* from mi

union all

select * 
from cva)

select distinct * from union_mace
order by person_id, union_mace.dod)